# LangChain with template and parser


将文本形成对象变量，便于模块化组装模板、形成条件控制结构、指定解析输出格式

1. create `ChatPromptTemplate` from plain_text_template, it detect named =={variables}==

2. then `ChatPromptTemplate` formats with user message to generate final prompt and send to LLM.

3. `Parser` generates format instructions with descriptive `Schema`s to ask LLM format chat response as JSON etc.

In [2]:
import os
from langchain.chat_models import ChatOpenAI

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())  # read local .env file

chat = ChatOpenAI(openai_api_key=os.environ['OPENAI_API_KEY'])
chat

In [19]:
customer_review ="""\
This leaf blower is pretty amazing. It has four settings:\
candle blower, gentle breeze, windy city, and tornado.\
It arrived in two days, just in time for my wife's anniversary present.\
I think my wife liked it so much she was speechless.\
So far I've been the only one using it, and I've been using it every other morning to clear the leaves.\
It's slightly more expensive than the other leaf blowers out there, \
but I think it's worth it for the extra features
"""
review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.
delivery days: How many days did it take for the product to arrive? if this infomation is not found, output -1.
price value: Extract any sentences about the value or price, and output them as a comma seperated python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text {text}
"""

In [20]:
from langchain.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] output_parser=None partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], output_parser=None, partial_variables={}, template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\ndelivery days: How many days did it take for the product to arrive? if this infomation is not found, output -1.\nprice value: Extract any sentences about the value or price, and output them as a comma seperated python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext {text}\n', template_format='f-string', validate_template=True), additional_kwargs={})]


ReAct: Thought-Action-Observation

In [14]:
{
    "gift": False,
    "delivery_days": 5,
    "price_value": "pretty affordable!"
}


{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [22]:
messages = prompt_template.format_messages(text=customer_review)
response = chat(messages)
print(response.content)
type(response.content)

{
  "gift": true,
  "delivery_days": 2,
  "price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features"]
}


In [ ]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser


gift_schema = ResponseSchema(name="gift", description="Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days", description="How many days did it take for the product to arrive? if this infomation is not found, output -1.")
price_value_schema = ResponseSchema(name="price_value", description="Extract any sentences about the value or price, and output them as a comma seperated python list.")

response_schemas = [gift_schema, delivery_days_schema, price_value_schema]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

In [35]:
review_template_v2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.
delivery days: How many days did it take for the product to arrive? if this infomation is not found, output -1.
price value: Extract any sentences about the value or price, and output them as a comma seperated python list.

text {text}

{format_instructions}
"""

prompt_template = ChatPromptTemplate.from_template(review_template_v2)
print(prompt_template)

messages = prompt_template.format_messages(text=customer_review, format_instructions=format_instructions)
response = chat(messages)

output_dict = output_parser.parse(response.content)
print(output_dict)
type(output_dict)

input_variables=['text', 'format_instructions'] output_parser=None partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['format_instructions', 'text'], output_parser=None, partial_variables={}, template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\ndelivery days: How many days did it take for the product to arrive? if this infomation is not found, output -1.\nprice value: Extract any sentences about the value or price, and output them as a comma seperated python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext {text}\n\n{format_instructions}\n', template_format='f-string', validate_template=True), additional_kwargs={})]
{'gift': False, 'delivery_days': 2, 'price_value': ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the 

dict

# Memories

`ConversationChain` can specify `Memory` objects which add historical responses back into prompts. `Memory` can have different constraints.

In [39]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
llm = ChatOpenAI(temperature=0.0)
memory = ConversationBufferMemory()
conversation = ConversationChain(llm=llm, memory=memory, verbose=True)


In [40]:
conversation.predict(input="hi, my name is Andrew")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: hi, my name is Andrew
AI:

> Finished chain.


"Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?"

In [41]:
conversation.predict(input="what is 1+1")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: hi, my name is Andrew
AI: Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?
Human: what is 1+1
AI:

> Finished chain.


'The answer to 1+1 is 2.'

In [42]:
conversation.predict(input="what is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: hi, my name is Andrew
AI: Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?
Human: what is 1+1
AI: The answer to 1+1 is 2.
Human: what is my name?
AI:

> Finished chain.


'Your name is Andrew, as you mentioned earlier.'

In [46]:
print(memory.buffer)

Human: hi, my name is Andrew
AI: Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?
Human: what is 1+1
AI: The answer to 1+1 is 2.
Human: what is my name?
AI: Your name is Andrew, as you mentioned earlier.
Human: hi
AI: what's up?


In [44]:
memory.load_memory_variables({})

{'history': "Human: hi, my name is Andrew\nAI: Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?\nHuman: what is 1+1\nAI: The answer to 1+1 is 2.\nHuman: what is my name?\nAI: Your name is Andrew, as you mentioned earlier."}

In [45]:
memory.save_context({"input":"hi"},{"output":"what's up?"})

In [47]:
print(memory.buffer)

Human: hi, my name is Andrew
AI: Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?
Human: what is 1+1
AI: The answer to 1+1 is 2.
Human: what is my name?
AI: Your name is Andrew, as you mentioned earlier.
Human: hi
AI: what's up?


In [ ]:
# Only keep last several (k) rounds of conversation history
from langchain.memory import ConversationBufferWindowMemory
window_memory = ConversationBufferWindowMemory(k=10)


# limit conversation history by tokens
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI
llm = ChatOpenAI(temperature=0.0)
token_memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=50)

from langchain.memory import ConversationSummaryBufferMemory
summary = ConversationSummaryBufferMemory(llm=llm, max_token_limit=50)
memory.save_context({"input":"hi"},{"output":"what's up?"})

# Chains

- LLMChain is the simplest.
- SimpleSequentialChain: single input/output
- SequentialChain: multiple input/output
- RouterChain: conditional branching



## LLMChain(Simple)

In [ ]:
import pandas as pd
df = pd.read_csv('data.csv')
df = head()

In [22]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())  # read local .env file

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI(openai_api_key=os.environ['OPENAI_API_KEY'], temperature=0.7)
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe a company that makes {product}")

chain = LLMChain(llm=llm, prompt=prompt)


In [23]:
product = "Queen Size Sheet Set"
chain.run(product)

'"Royal Linens"'

## SequentialChain

In [24]:
from langchain.chains import SequentialChain

llm = ChatOpenAI(openai_api_key=os.environ['OPENAI_API_KEY'], temperature=0.7)

#prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    'Translate the following review to English:'
    '\n\n{Review}')
# chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key='English_Review')

#prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    'Can you summerize the following review in 1 sentence:'
    '\n\n{English_Review}')
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key='summary')

#prompt template 3
third_prompt = ChatPromptTemplate.from_template(
    'What language is the following review:'
    '\n\n{Review}')
# chain 3
chain_three = LLMChain(llm=llm, prompt=third_prompt, output_key='language')

#prompt template 4
fourth_prompt = ChatPromptTemplate.from_template(
    'Write a follow up response to the following '
    'summary in the specified language:'
    '\n\nSummary: {summary} \n\nLanguage: {language}')
# chain 4
chain_four = LLMChain(llm=llm, prompt=fourth_prompt, output_key='followup_message')

overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary", "followup_message"]    )

In [26]:
review = "Je trouve le gout mediocre. La mousse ne tient pas, c'est bizarre. j'achete les memes dans le commerceet le got est bien meilleur...\nVieux lot ou contrefacon!?"
overall_chain(review)

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-TCk9oFMNBeKhkGqOBQwYov4Z on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-TCk9oFMNBeKhkGqOBQwYov4Z on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht

KeyboardInterrupt: 

## RouterChain
1. AI decides candidate prompt according to user input, like a conditional branch
2. chat with the chosen prompt, or else the default one

In [69]:
physics_template = """"You are a very smart phvsiccs professor.\
You are great at answering questions about physics in a concret way.\
When you don't know the answer to a question you admit that.

Here is a question:
{input}
"""
math_template = """you are a very good mathematician. You are so good because you are able to break down hard \
problem and answer the component parts, and then put them together to an answer.

Here is a question:
{input}
"""

history_template = """you are a very good historian. You have contexts from a range of historical periods.\
You have the ability to evaluate the past. You have a respect for historical evidence to support \
your explanations and judgements.

Here is a question:
{input}
"""

computerscience_template = """You are a successful computer scientist. You have a passional of \
forward-thinking, confidence，strong problem-solving capability and excellent communication skills,\
You are great at answering. You are so good because you know how to solve a problem by breaking down so that\
a machine can easily interpret and you know how to choice time complexity and space complexity.

Here is a question:
{input}
"""

In [70]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics.",
        "prompt_template": physics_template
    },
    {
        "name": "math",
        "description": "Good for answering math questions.",
        "prompt_template": math_template
    },
    {
        "name": "history",
        "description": "Good for answering history questions.",
        "prompt_template": history_template
    },
    {
        "name": "computer science",
        "description": "Good for answering computer science questions.",
        "prompt_template": computerscience_template
    },
]

In [71]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import PromptTemplate


In [72]:
llm = ChatOpenAI(openai_api_key=os.environ['OPENAI_API_KEY'], temperature=0.7)

In [73]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain
    
# print(destination_chains)

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
print(destinations_str)

physics: Good for answering questions about physics.
math: Good for answering math questions.
history: Good for answering history questions.
computer science: Good for answering computer science questions.


In [74]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [90]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a language model select the model prompt best suited for the input. 
You will be given the names of the available prompts and a description of what the prompt is best suited for.
You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.\n\n


<< FORMATTING>>
Return a markdown code snippet with a JSON object formatted to look like:\n
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"\n
    "next_inputs": string \ a potentially modified version of the original input\n
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.\n\n

<< CANDIDATE PROMPTS>>
{destinations}


<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>
again, "destination" the ```json MUST be one of the candidate prompt names specified below OR it must be "DEFAULT" if the input is not well suited for any of the candidate prompts.
"""

In [91]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),)

router_chain = LLMRouterChain.from_llm(llm, router_prompt, verbose=True)
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. 
You will be given the names of the available prompts and a description of what the prompt is best suited for.
You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.




<< FORMATTING>>
Return a markdown code snippet with a JSON object formatted to look like:

```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"

    "next_inputs": string \ a potentially modified version of the original input

}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.



<< CANDIDATE PROMPTS>>
physics: Good for answering questions about physics.
math: Good for answering math questions.


In [92]:
chain = MultiPromptChain(router_chain=router_chain,
                destination_chains=destination_chains,
                default_chain=default_chain,
                verbose=True)
# print(chain)

In [43]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'black body radiation'}
> Finished chain.


'Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an object that absorbs all radiation incident upon it and emits radiation at all wavelengths and frequencies according to its temperature. This radiation is characterized by a continuous spectrum, meaning it contains all wavelengths and frequencies, and its intensity is determined by the temperature of the black body. The concept of black body radiation is important in understanding many phenomena in physics, including the behavior of stars, the operation of incandescent light bulbs, and the greenhouse effect.'

In [64]:
chain.run("What is 2+2?")



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
math: {'input': 'What is 2+2?'}

> Entering new LLMChain chain...
Prompt after formatting:
Human: you are a very good mathematician. You are so good because you are able to break down hard problem and answer the component parts, and then put them together to an answer.

Here is a question:
What is 2+2?


> Finished chain.

> Finished chain.


'The answer to the question "What is 2+2?" is 4.'

In [93]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
None: {'input': 'Why does every cell in our body contain DNA?'}

> Entering new LLMChain chain...
Prompt after formatting:
Human: Why does every cell in our body contain DNA?

> Finished chain.

> Finished chain.


'Every cell in our body contains DNA because DNA carries the genetic information that determines the characteristics and functions of each cell. This genetic information is necessary for the growth, development, and proper functioning of the body. DNA also ensures that genetic information is passed on from one generation to the next, which is essential for the perpetuation of life.'